In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install --upgrade torch==2.0.0 torchvision

In [ ]:
# !pip install apache_beam mwparserfromhell
# !pip install datasets

In [2]:
# Helper scripts from github
!pip install --force-reinstall https://github.com/pfornia/paul-gpt/blob/master/dist/paul_gpt-0.0.1-py3-none-any.whl?raw=true

  Using cached https://github.com/pfornia/paul-gpt/blob/master/dist/paul_gpt-0.0.1-py3-none-any.whl?raw=true (5.7 kB)
  Attempting uninstall: paul-gpt
    Found existing installation: paul_gpt 0.0.1
    Uninstalling paul_gpt-0.0.1:
      Successfully uninstalled paul_gpt-0.0.1


In [3]:
## public libraries
from datasets import load_dataset
import torch
import datetime

## local libraries
from paul_gpt.gpt_utils import (
# from gpt_utils import (    # LOCAL VERSION
    get_encoder_decoder_size,
    text_to_tv_tensors,
    get_batch,
    training_run,
    test_forward_pass,
    test_gen_text
)
from paul_gpt.attention_decoder import AttentionModule 
# from attention_decoder import AttentionModule  # LOCAL VERSION


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.set_default_device(device)
device

device(type='cpu')

In [5]:
wiki_raw = load_dataset("wikipedia", "20220301.simple")

Found cached dataset wikipedia (/Users/paul/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
wiki_raw

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 205328
    })
})

In [7]:
wiki_text_blob = '\n\n'.join(wiki_raw['train']['text'])
print(len(wiki_text_blob))
print()
print(wiki_text_blob[:1000])

215900536

April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.

April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.

April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.

The Month 

April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.

April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.

In common years, April starts on the same day of the week as October of the previous year, and in le

In [8]:
##When sorted, low-numbered codes seem to be more "normal"
normal_chars = ''.join(sorted(list(set(wiki_text_blob)))[0:97])
normal_chars

'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [9]:
# 30 sec ish
wiki_text_blob_clean = ''.join([x for x in wiki_text_blob if x in normal_chars])

In [10]:
# Keep 99.7% of chars. great.
len(wiki_text_blob_clean)/len(wiki_text_blob)

0.997293827931951

In [11]:
encode, decode, vocab_size = get_encoder_decoder_size(wiki_text_blob_clean)
print(vocab_size)

97


In [12]:
train, validate = text_to_tv_tensors(wiki_text_blob_clean, encode, device)

In [13]:
m_attn = AttentionModule(vocab_size).to(device)

In [ ]:
test_forward_pass(m_attn, validate)

In [14]:
!ls ../model_checkpoints

m_attn_2023-05-01_2200.pt       m_attn_2023-05-01_2200_10000.pt


In [15]:
m_attn.load_state_dict(torch.load('../model_checkpoints/m_attn_2023-05-01_2200_10000.pt', map_location=torch.device('cpu')))
_ = m_attn.eval()

In [ ]:
# CPU = ~25s per epoch
# GPU ~ 1s per epoch (or 15 min for 1000)

now = datetime.datetime.now()
print("Start: " + str(now))
training_run(m_attn, train, validate, num_epochs=1)
print("Runtime: " + str(datetime.datetime.now() - now))

In [17]:
# Single inference on CPU: ~2min
seed_raw = """
Tonight I'll dream, while I'm in bed
When silly thoughts go through my head
About the bugs and alphabet
And when I wake tomorrow, I'll bet
That you and I will walk together again

I can tell that we are gonna be friends
"""

test_gen_text(m_attn, seed_raw, encode, decode, device)


                                   
Tonight I'll dream, while I'm in bed
When silly thoughts go through my head
About the bugs and alphabet
And when I wake tomorrow, I'll bet
That you and I will walk together again

I can tell that we are gonna be friends
Like Edmonton, an issue coward-wheel, a vibwe length
Standhson, vibroting
However, a town of wood, and extras. It holds to perform like the professor of their work with vibroti, and extrassion like it wise after grammotime, earned by the distance in the eastern state point. He led to play for the professor to the last send as his book. Her following back and maker against the man it was looked by the stories, him to hone of a kind of his acting.

After the 2000s, horse shows Brian Faster's cousing will be shows. It is happing in exposed materials of the US priest are by Michael. It was now item in Brian County.

Counties in Ocean region in Stockholm, California in the U.S. state of Skyller-President at the United States.

His counties

4000 epochs version on May 1st (loss ~1.4):

[seed start]
Tonight I'll dream, while I'm in bed
When silly thoughts go through my head
About the bugs and alphabet
And when I wake tomorrow, I'll bet
That you and I will walk together again

I can tell that we are gonna be friends
[seed end]

Living people

Germany dons the organisan languages
Kolelly High Skyne (196821), where includes were lungth state.  It is in the soation of In of the Great Official Caugue is state.
 Living pound episode in Pardy's anguage is the player of Yaskorp Official Party.
 Brough Indias Republican: single 19 August 2012 of the UK airported Free of Indias.

In a life original coast her reduced in Kasao, Bunitania and the Kambridgy.
 Many Lenso (d. 2016), Walkorfern, Texas and Punnus of Kapashi, Punja, Elizar Peak, Texas, France and Kambridgy, United Kapashi of travel. The population of Bus. All Quicka, Karal, Keapau, Virginian, Baki freemat Joe Lenson, Tenasyas, which did days not the Role and North Africa Empire from the AMD
 Game Operson and United Kingdom Awards
 Game The Flate of Carol.
 A contrown of the Game's Met Des of Game
 Fleenh Zalan, desting name ankwards
 Village Texas Cash be not ask lister
 Am When Hit Rebelle, control, cameral be usually singles standards (when two comparates)


 10000 epochs version on May 1st (loss 1.2):

 [seed, plus:]

 Like Edmonton, an issue coward-wheel, a vibwe length
Standhson, vibroting
However, a town of wood, and extras. It holds to perform like the professor of their work with vibroti, and extrassion like it wise after grammotime, earned by the distance in the eastern state point. He led to play for the professor to the last send as his book. Her following back and maker against the man it was looked by the stories, him to hone of a kind of his acting.

After the 2000s, horse shows Brian Faster's cousing will be shows. It is happing in exposed materials of the US priest are by Michael. It was now item in Brian County.

Counties in Ocean region in Stockholm, California in the U.S. state of Skyller-President at the United States.

His counties

References

1945 births
2014 deaths
Azerbaijani disestablishments
California Presidents
Swedish politicians

Azerbaijani Afango (born September 8, 1965) is an American professional aathlic team. Ribbet was born in Bangladesh, Texas. Rupillo Anatho Presid